# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [3]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data   


Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [4]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE

class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

class RB_Node(Node):
    RED = True
    BLACK = False

    def __init__(self, value, colour):
        super().__init__(value)
        self.colour = colour

Use the cell below to implement the requested API by means of **sequential search**.

In [5]:
class SequentialSearchSet(AbstractSet):
    def __init__(self):
        self.set = []      
     
    def insertElement(self, element):
        if self.searchElement(element) is False:
            self.set.append(element)
            return True
        return False
    
    def searchElement(self, element):
        for member in self.set:
            if member == element:
                return True
        return False

Use the cell below to implement the requested API by means of **binary search tree**.

In [6]:
class BinarySearchTreeSet(AbstractSet):
    def __init__(self):
        self.root = None
    
    def insertElement(self, element):
        if not self.root:
            self.root = Node(element)
            return True
        else:
            current = self.root
            while current:
                if current.value == element:
                    return False
                elif current.value > element:
                    if not current.left:
                        current.left = Node(element)
                        return True
                    current = current.left
                elif current.value < element:
                    if not current.right:
                        current.right = Node(element)
                        return True
                    current = current.right
        return False

    def searchElement(self, element):
        current = self.root
        while current:
            if current.value == element:
                return True
            elif current.value > element:
                current = current.left
            elif current.value < element:
                current = current.right
        return False


Use the cell below to implement the requested API by means of **balanced search tree**.

In [7]:
class BalancedSearchTreeSet(AbstractSet):
    def __init__(self):
        self.root = None

    @staticmethod
    def is_red(node):
        if node is None:
            return False
        return node.colour == RB_Node.RED

    @staticmethod
    def rotate_left(node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = RB_Node.RED
        return x

    @staticmethod
    def rotate_right(node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = RB_Node.RED
        return x

    @staticmethod
    def flip_colours(node):
        node.colour = RB_Node.RED
        node.left.colour = RB_Node.BLACK
        node.right.colour = RB_Node.BLACK
    
    def insertElement(self, element):
        self.root, inserted = self._insertElement(self.root, element)
        self.root.colour = RB_Node.BLACK
        return inserted
    
    def _insertElement(self, node, element):
        if node is None:
            return RB_Node(element, RB_Node.RED), True

        if element < node.value:
            node.left, inserted = self._insertElement(node.left, element)
        elif element > node.value:
            node.right, inserted = self._insertElement(node.right, element)
        else:
            return node, False
        
        if self.is_red(node.right) and not self.is_red(node.left):
            node = self.rotate_left(node)
        if self.is_red(node.left) and self.is_red(node.left.left):
            node = self.rotate_right(node)
        if self.is_red(node.left) and self.is_red(node.right):
            self.flip_colours(node)
        return node, inserted    

    def searchElement(self, element):     
        return self._searchElement(self.root, element) == element

    def _searchElement(self, node, element):
        if node is None:
            return False

        if element < node.value:
            return self._searchElement(node.left, element)
        elif element > node.value:
            return self._searchElement(node.right, element)
        else:
            return node.value   

Use the cell below to implement the requested API by means of **bloom filter**.

In [8]:
from bitarray import bitarray

EXPECTED_NUM_INSERTIONS = 500000
BITARRAY_SIZE = EXPECTED_NUM_INSERTIONS * 50
LN2 = 0.69314718056

class BloomFilterSet(AbstractSet):
    def __init__(self, size, num_hashes=3):
        self.size = size
        self.num_hashes = num_hashes
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)

    def _hash(self, element):
        hash_values = []
        for i in range(self.num_hashes):
            hash_value = hash(f'{element}{i}')
            hash_values.append(hash_value % self.size)
        return hash_values
    
    @staticmethod
    def optimise_k(m, n):
        return 1 + int(LN2 * (m / n))
        
    def insertElement(self, element):
        if self.searchElement(element) is False:
            hash_values = self._hash(element)
            for value in hash_values:
                self.bit_array[value] = 1
            return True
        else:
            return False
    
    def searchElement(self, element):     
        hash_values = self._hash(element)
        for value in hash_values:
            if not self.bit_array[value]:
                return False
        return True

Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [9]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    NUM_WORDS = 50000
    MIN_LENGTH = 1
    AV_LOW_LENGTH = 2
    AV_HIGH_LENGTH = 6
    MAX_LENGTH = 12
    CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
    VOWELS = 'aeiou'
    test_file = "synthetic_test_search.txt"
    SAMPLE_INTERVAL = 200
    test_words = set()

    def __init__(self):
        pass

    def generate_word(self, scarcity=0.05):
        limit = int(1 / scarcity)
        roulette = random.randint(0, int(1 / scarcity))
        if roulette == limit:
            length = random.randint(self.AV_LOW_LENGTH, self.AV_HIGH_LENGTH)
        else:
            length = random.randint(self.MIN_LENGTH, self.MAX_LENGTH)
        word = ''
        for i in range(length):
            if i % 2 == 1 or length == 1:
                word += random.choice(self.VOWELS)
            else:
                word += random.choice(self.CONSONANTS)
        return word

    # Generate the words and put them in an array + create a test file
    def generateData(self, size):
        data = []
        for i in range(size):
            word = self.generate_word()
            data.append(word)
            if i % self.SAMPLE_INTERVAL == 0:
                self.test_words.add(word)
                self.test_words.add(self.generate_word())

        test_words = list(self.test_words)
        with open(self.test_file, 'w') as f:
            for i in range(len(test_words)):
                f.write(test_words[i])
                if i <= len(test_words):
                    f.write('\n')

        return data



Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [12]:
 # ADD YOUR TEST CODE HERE TO WORK ON REAL DATA
import timeit
import random
import string

HASH_KEYS = 5

@staticmethod
def txt_to_list(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
        if (file_path == "test-search.txt") or (file_path == "synthetic_test_search.txt"):
            return content.split("\n")
        else:
            return content.split()

class ExperimentalFramework:
    def __init__(self, data_file, data_structures_dict, operation_repeats=50, search_intervals=5, search_test_file="test-search.txt"):
        self.data_values = txt_to_list(data_file)
        self.search_words = txt_to_list(search_test_file)
        self.data_structures_dict = data_structures_dict
        self.operation_repeats = operation_repeats
        self.intervals = len(self.data_values) // search_intervals
        self.insert_time = {}
        self.mean_search_time = {}

    def calc_insert_search_time_multiple_data_structures(self):
        for data_structure_name, data_structure in self.data_structures_dict.items():
            print(data_structure_name)
            self.calc_insert_search_time_one_data_structure(data_structure_name, data_structure)

    def calc_insert_search_time_one_data_structure(self, data_structure_name, data_structure):
        self.insert_time[data_structure_name] = {}
        self.mean_search_time[data_structure_name] = {}

        for i, word in enumerate(self.data_values):
            self.insert_time[data_structure_name][i] = timeit.timeit(lambda: data_structure.insertElement(word), number=self.operation_repeats)

            if i % self.intervals == 0:
                # Executes the search function in specified intervals for the data structure over the elements already input
                search_time = [timeit.timeit(lambda: data_structure.searchElement(word), number=self.operation_repeats) for word in self.search_words]
                self.mean_search_time[data_structure_name][i] = sum(search_time) / (self.operation_repeats * len(search_time))
        print(f"Mean Search Time - {self.mean_search_time}")

    def get_insert_search_time_one_data_structure(self):
        return self.calc_insert_search_time_one_data_structure(list(data_structures_dict)[0], list(data_structures_dict.values())[0])


class RunTests(ExperimentalFramework):

    def __init__(self, data_file, data_structures_dict, operation_repeats=50, search_intervals=5, search_test_file="test-search.txt"):
        super().__init__(data_file, data_structures_dict, operation_repeats, search_intervals, search_test_file)

    def sorted_real_data(self):
        self.data_values = sorted(self.data_values)
        self.calc_insert_search_time_multiple_data_structures()

    def no_repeats_real_data(self):
        self.data_values = set(self.data_values)
        print(self.data_values)
        self.calc_insert_search_time_multiple_data_structures()

    def normal_real_data(self):
        self.calc_insert_search_time_multiple_data_structures()

if __name__ == '__main__':
    data_structures_dict = {"BST": BinarySearchTreeSet()}
    test = RunTests("test1-mobydick.txt", data_structures_dict, 1, 100)
    test.normal_real_data()

BST
Mean Search Time - {'BST': {0: 2.1972488441051694e-06, 2093: 1.0801650105266395e-05, 4186: 1.2429724789198933e-05, 6279: 1.8945871425126123e-05, 8372: 1.3963670529630206e-05, 10465: 2.1063120941708394e-05, 12558: 1.3487523319114239e-05, 14651: 1.6987522948670006e-05, 16744: 1.3984219649584469e-05, 18837: 1.4412476693989214e-05, 20930: 1.6676696282256086e-05, 23023: 1.567945102178449e-05, 25116: 1.6670825745944584e-05, 27209: 1.7515229116783503e-05, 29302: 1.7584954738753652e-05, 31395: 1.701027488147994e-05, 33488: 1.6526053467372417e-05, 35581: 1.669247867539525e-05, 37674: 1.8517613526297e-05, 39767: 2.1158532162598513e-05, 41860: 1.7886789549791484e-05, 43953: 1.6159081934105367e-05, 46046: 1.6832842658179888e-05, 48139: 1.8337063615235988e-05, 50232: 1.715908410164331e-05, 52325: 1.784238660998574e-05, 54418: 1.4138347234303525e-05, 56511: 1.7141468327389944e-05, 58604: 2.0007341523406977e-05, 60697: 1.851082512459487e-05, 62790: 1.8433944339257316e-05, 64883: 1.801853324500245

In [13]:
# ADD YOUR TEST CODE HERE TO WORK ON SYNTHETIC DATA
import timeit

NUM_SYNTHETIC = 100000
HASH_KEYS = 5

def list_to_txt(list):
    with open("synthetic_data.txt", "w") as file:
        for value in list:
            file.write(value)
            file.write('\n')

class SyntheticData(ExperimentalFramework):
    def __init__(self, data_structures_dict, operation_repeats=50, search_intervals=5):
        self.data_values = TestDataGenerator().generateData(NUM_SYNTHETIC)
        list_to_txt(self.data_values)
        self.data_file = "synthetic_data.txt"
        self.search_test_file = "synthetic_test_search.txt"
        super().__init__(self.data_file, data_structures_dict, operation_repeats, search_intervals, self.search_test_file)

    def get_results(self):
        self.calc_insert_search_time_multiple_data_structures()

if __name__ == '__main__':
    data_structures_dict = {"BF": BloomFilterSet(BITARRAY_SIZE, HASH_KEYS)}
    test_synthetic = SyntheticData(data_structures_dict, 10, 100)
    test_synthetic.get_results()




BF
Mean Search Time - {'BF': {0: 1.1083307232468404e-05, 1000: 1.3630089308421373e-05, 2000: 1.3276122864231717e-05, 3000: 1.3906893908311535e-05, 4000: 1.3283575441448965e-05, 5000: 1.419188816350099e-05, 6000: 1.0463742906092432e-05, 7000: 1.3725240162037855e-05, 8000: 1.1369061465088965e-05, 9000: 1.2196860342499846e-05, 10000: 5.9843574281178375e-06, 11000: 1.2999083820792882e-05, 12000: 1.2954703893762228e-05, 13000: 1.2959955336900646e-05, 14000: 1.3381966445647054e-05, 15000: 1.4834748568044528e-05, 16000: 1.3354782089226822e-05, 17000: 1.2139117308793454e-05, 18000: 1.229715091306404e-05, 19000: 6.092279335202951e-06, 20000: 1.2917552928976173e-05, 21000: 1.3219832531150493e-05, 22000: 1.3152134055844066e-05, 23000: 1.270620110730885e-05, 24000: 1.30410501460269e-05, 25000: 1.2973720752816211e-05, 26000: 1.3293162022077337e-05, 27000: 1.3130312920782153e-05, 28000: 1.3242044639591565e-05, 29000: 1.2831385318949485e-05, 30000: 1.3287240234801973e-05, 31000: 1.2816536382059229e-0